In [ ]:
pwd

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

In [2]:
p = 2
n = 100
k = 4

DG = DatasetGenerator(p=p, N=n, K=k, class_task=True)

feature_means = np.array([[5,5], [-5,-5], [-3,2], [5,-1]])
DG.generate_minimal_data(feature_means, np.ones((k,p)))
for k_ in range(k):
    plt.scatter(DG.X[0, DG.y[k_]==1], DG.X[1, DG.y[k_]==1])

NameError: name 'DatasetGenerator' is not defined

In [ ]:
p = 1
k = 2
DG = DatasetGenerator(p=p, N=n, K=k, class_task=False)

feature_means = np.array([5, 5])
feature_std = np.ones(p)
regression_rule = np.array([[-3, 1]])
DG.generate_minimal_data(feature_means, feature_std, regression_rule=regression_rule)

for k_ in range(k):
    plt.plot(DG.X[0], DG.y[0])    
    plt.plot(DG.X[0], DG.y[1])

In [ ]:
p = 2
k = 1
DG = DatasetGenerator(p=p, N=n, K=k, class_task=False)

feature_means = np.array([10, 5])
feature_std = np.ones(p)
regression_rule = np.array([[-3, 1]])
DG.generate_minimal_data(feature_means, feature_std, regression_rule=regression_rule)


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(DG.X[0], DG.X[1], DG.y) 

In [ ]:
X_t = DG.add_redundancy(add_p=10)

In [ ]:
X_t.shape, np.linalg.matrix_rank(X_t)

In [ ]:
X_t = DG.add_redundancy(A=np.random.randn(3,p))
print(X_t.shape)
print(np.linalg.matrix_rank(X_t))

In [ ]:
X_t = DG.add_noise(add_p=10)
print(X_t.shape, np.linalg.matrix_rank(X_t))
print([np.mean(X_t_feat) for X_t_feat in X_t])
print([np.std(X_t_feat) for X_t_feat in X_t])

In [ ]:
X_t = DG.add_noise(add_p=10, noise_mean=np.ones(10), noise_std=np.ones(10))
print(X_t.shape, np.linalg.matrix_rank(X_t))
print([np.mean(X_t_feat) for X_t_feat in X_t])
print([np.std(X_t_feat) for X_t_feat in X_t])

In [67]:
class DatasetGenerator:

    def __init__(self,
                 p,
                 N,
                 K,
                 class_task=False,
                 scenario=1,
                 mu_array=None,
                 sigma_array=None,
                 **kwargs):
        """
        Generate dataset for a supervised learning task. The features are extracted using
        Gaussian distributions.
        :param p: int, of relevant features
        :param N: int, number of data
        :param K: int, output dimension for regression, number of classes for classification task
        :param class_task: bool, if False regression, else classification dataset
        :param scenario: int, scenario (1,2,4)
        :param mu_array: np.array (self.p) of means if regression, otherwise
        (self.K, self.p) array for classification
        :param sigma_array: np.array (self.p) of standard deviations, otherwise
        (self.K, self.p) array for classification
        :param regression_rule: only if self.class_task is False, we need a regression law
        """
        self.p = p
        self.N = N
        self.K = K
        self.class_task = class_task
        self.scenario = scenario
        self.mu_array = mu_array
        self.sigma_array = sigma_array

        self.X = None
        self.y = None
        self.minimal_dataset = None
        
        self.generate_minimal_data()  # we generate the minimal dataset given the hyper-parameters

        if scenario is not None:
            if 'add_p' not in dct_kwargs.keys():
                raise ValueError("Specify the amount of additional features through add_p")
            elif dct_kwargs['add_p'] == 0:
                return
            self.add_p = dct_kwargs['add_p']
        
        if self.scenario == 1:  # additional noisy features
            self.noise_mean = np.array(dct_kwargs['noise_mean']) if 'noise_mean' in dct_kwargs.keys() else 0
            self.noise_std = np.array(dct_kwargs['noise_std']) if 'noise_std' in dct_kwargs.keys() else 1
            self.add_gaussian_noise()
            
        elif self.scenario == 2:  # additional redundant features
            self.A = np.array(dct_kwargs['A']) if 'A' in dct_kwargs.keys() else None
            self.add_redundancy()
            
        elif self.scenario == 4:  # add noise and redundancy
            self.redundancy_amount = dct_kwargs['redundancy_amount'] if 'redundancy_amount' in dct_kwargs.keys() else 0.5
            self.A = np.array(dct_kwargs['A']) if 'A' in dct_kwargs.keys() else None
            self.noise_mean = np.array(dct_kwargs['noise_mean']) if 'noise_mean' in dct_kwargs.keys() else 0
            self.noise_std = np.array(dct_kwargs['noise_std']) if 'noise_std' in dct_kwargs.keys() else 1
            # when we call one or the other we automatically save the new self.X_transf
            self.add_mixture()
        self.dct_kwargs = kwargs


    def generate_minimal_data(self):
        """
        Here we generate the data by using the relevant features only.
        Each feature is Gaussianly distributed. Mean and standard
        deviation for each variable varies depending on the user specification.

        The generic i-th feature is x_i
                    x_i = mean_i + N(0,1) * std_i, x_i in R^n_samples

        The labels are generating depending on the learning task.
        If class_task is False, a regression rule is needed, and
            y_k = np.dot(regression_rule, x_k)  , x_k in R^n_features,

        If class_task is True, the classifier the two distribution are
        given different values. # at the moment we are not considering the
        multiclassification case.
        """
        if not self.class_task and self.regression_rule is None:  # for regression task
            raise ValueError("You need a learning rule to generate the regression law.")

        # regression task
        if not self.class_task:
            self.regression_rule = np.squeeze(self.regression_rule)

            if self.regression_rule.ndim == 1:  #  p or K are equal to one
                if self.regression_rule.size != self.p and self.regression_rule.size != self.K:
                    raise ValueError("The regression rule does not match the data dimensionality")
            else:  # if the dimensionality if bigger than one
                check_output, check_input = self.regression_rule.shape
                if check_output != self.K or check_input != self.p:
                    raise ValueError("The dimensions for the regression rule do not match the data")

            X_ = np.random.randn(self.p, self.N)  # we generate the new dataset
            for id_, (mu_, sigma_) in enumerate(zip(self.mu_array, self.sigma_array)):
                X_[id_] *= sigma_
                X_[id_] += mu_
            self.X = X_
            if self.K != 1 and self.p == 1:  # if the number of output != 1
                self.regression_rule = self.regression_rule.reshape(-1, 1)
            elif self.K == 1 and self.p != 1:
                self.regression_rule = self.regression_rule.reshape(-1, )
            self.y = np.dot(self.regression_rule, self.X)

        # classification task
        else:
            check_output_mu, check_input_mu = np.squeeze(self.mu_array).shape
            check_output_st, check_input_st = np.squeeze(self.sigma_array).shape

            if check_output_mu != self.K or check_output_st != self.K:
                raise ValueError("Arrays inconsistent with the number of classes")

            n_per_class = self.N // self.K

            X_ = np.zeros((self.p, self.N))
            y_ = np.zeros((self.K, self.N))
            for k_, (mu_class_, sigma_class_) in enumerate(zip(self.mu_array, self.sigma_array)):  # for each class
                first_ = k_ * n_per_class
                last_ = self.N if k_ == self.K - 1 else (k_ + 1) * n_per_class
                for id_, (mu_, sigma_) in enumerate(zip(mu_class_, sigma_class_)):
                    X_[id_, first_:last_] = mu_ + np.random.randn(last_ - first_) * sigma_
                y_[k_, first_:last_] = 1

            self.y = y_
            self.X = X_
        self.minimal_dataset = True

        return self
    
    def add_redundancy(self):
        """ We add redundancy to the dataset.
        Using a linear combination of the input features.
        """
        if not self.minimal_dataset:
            raise ValueError("Generate the dataset first")

        if self.A is not None:
            check_add_feat, check_d = self.A.shape  # check the dimensions
            if check_d != self.p:
                raise ValueError("The dimension of the transformation does not check the amount of features")

        else:
            self.A = np.random.randn(self.add_p, self.p)  #  add redundancy through a linear transformation

        self.X_transf = np.vstack((self.X, self.A.dot(self.X)))  # save as an attribute the feature
        
    
    def add_gaussian_noise(self):
        """ We add noisy features to the dataset.
        This is done by adding gaussian distributed
        random variables to the original features.
        """
        if not self.minimal_dataset:
            raise ValueError("Generate the dataset first")

        if np.isscalar(self.noise_mean) and np.isscalar(self.noise_std):
            self.X_transf = np.vstack((self.X, (self.noise_mean + 
                                                self.noise_std * np.random.randn(self.add_p, self.N))))
            return
        
        if self.add_p != self.noise_mean.size and self.add_p != self.noise_std.size:
            raise ValueError("Mismatch in dimensions")
            
        noise_feat = np.zeros((self.add_p, self.N))
        for i_, (noise_m_, noise_s_) in enumerate(zip(self.noise_mean, self.noise_std)):
            noise_feat[i_] = noise_m_ + noise_s_ * np.random.randn(self.N)
        
        self.X_transf = np.vstack((self.X, noise_feat))

        return self
    
    def add_mixture(self):
        """With this call we add a percentage of redundancy and a (1-percentage) of noisy features.
        """
        if self.A is None and np.isscalar(self.noise_mean) and np.isscalar(self.noise_std):
            n_noise_feat = (self.add_p - int(self.add_p * self.redundancy_amount))
            n_rdndt_feat = int(self.add_p * self.redundancy_amount)
            self.A = np.random.randn(n_rdndt_feat, self.p)
            noise_feat = self.noise_mean + self.noise_std * np.random.randn(n_noise_feat, self.N)

        elif self.A is not None:
            n_rdndt_feat, p_tmp = self.A.shape
            n_noise_feat = self.add_p - n_rdndt_feat
            if p_tmp != self.p:
                raise ValueError("Mismatch in dimensions for the linear transformation")
                
            if not np.isscalar(self.noise_mean) and not np.isscalar(self.noise_std):
                noise_feat = np.zeros((self.noise_mean.size, self.N))
                if self.noise_mean.size != self.noise_std.size:  # check that the shapes are consistent
                    raise ValueError("The noise values are inconsistent in shape")
                n_noise_feat = self.noise_mean.size
                # check not to exceed the add_p value
                            
                if self.add_p != (n_noise_feat + n_rdndt_feat):
                    raise ValueError("The sum of the two dimensions is different from the specified number of additional features")
                for i_, (noise_m_, noise_s_) in enumerate(zip(self.noise_mean, self.noise_std)):
                    noise_feat[i_] = noise_m_ + noise_s_ * np.random.randn(self.N)
            else:
                # tested
                noise_feat = self.noise_mean + self.noise_std * np.random.randn(n_noise_feat, self.N)
            
        X_r = np.vstack((self.X, np.dot(self.A, self.X)))
        self.noise = noise_feat
        
        self.X_transf = np.vstack((X_r, noise_feat))

In [ ]:
mu_array = np.array([[5,5], [-5,-5], [-3,2], [5,-1]])
sigma_array = np.ones((4,2))
p = 2
n = 100
k = 4
add_p = 10 

DataGen = DatasetGenerator(p=p,N=n,K=k,class_task=True,
                           scenario=2, mu_array=mu_array,
                           sigma_array=sigma_array,
                           regression_rule=1,
                           add_p=add_p,
                           A=list(np.random.randn(add_p, p)))

for k_ in range(k):
    plt.scatter(DataGen.X[0, DataGen.y[k_]==1], 
                DataGen.X[1, DataGen.y[k_]==1])

In [ ]:
mu_array = np.array([[5,5], [-5,-5], [-3,2], [5,-1]])
sigma_array = np.ones((4,2))
p = 2
n = 100
k = 4
add_p = 10 

DataGen = DatasetGenerator(p=p,N=n,K=k,class_task=True,
                           scenario=1, mu_array=mu_array,
                           sigma_array=sigma_array,
                           add_p=add_p,
                           noise_mean=list(np.zeros(add_p)),
                           noise_std=list(3*np.ones(add_p)))

for k_ in range(k):
    plt.scatter(DataGen.X[0, DataGen.y[k_]==1], 
                DataGen.X[1, DataGen.y[k_]==1])

In [ ]:
# verify if you satify all the required conditions

corrupted_dataset = DataGen.X_transf
print(np.linalg.matrix_rank(corrupted_dataset))

mean_ = np.mean(corrupted_dataset, axis=-1)
std_ = np.std(corrupted_dataset, axis=-1)
for (m_, s_) in zip(mean_, std_):
    print(m_, s_)

In [ ]:
mu_array = np.array([[5,5], [-5,-5], [-3,2], [5,-1]])
sigma_array = np.ones((4,2))
p = 2
n = 100
k = 4
add_p = 10 

DataGen = DatasetGenerator(p=p,N=n,K=k,class_task=True,
                           scenario=4, mu_array=mu_array,
                           sigma_array=sigma_array,
                           add_p=add_p,
                           redundancy_amount=0.25,
                           A=list(np.random.randn(3,p)),
                           noise_mean=list(np.array([2,3,4,5,1,1,7])),
                           noise_std=list(np.ones(7)))

for k_ in range(k):
    plt.scatter(DataGen.X[0, DataGen.y[k_]==1], 
                DataGen.X[1, DataGen.y[k_]==1])

In [ ]:
DataGen.A, DataGen.noise.shape

In [ ]:
np.mean(DataGen.noise, axis=1)

In [ ]:
np.std(DataGen.noise, axis=1)

In [ ]:
dct_kwargs = {'add_p': 10,
              'redundancy_amount': 0.25,
              'A': list(np.random.randn(3,p)), 
              'noise_mean': list(np.array([2,3,4,5,1,1,7])),
              'noise_std': list(np.ones(7))}

In [ ]:
dct_kwargs = {'add_p': 0,
              'redundancy_amount': 0.25,
              'A': list(np.random.randn(3,p)), 
              'noise_mean': list(np.array([2,3,4,5,1,1,7])),
              'noise_std': list(np.ones(7))}

DataGen = DatasetGenerator(p=p,N=n,K=k,class_task=True,
                           scenario=4, mu_array=mu_array,
                           sigma_array=sigma_array,
                           **dct_kwargs)

DataGen.X.shape

# TensorFlow section

In [4]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from os.path import join
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [12]:
p = 3
k = 2

dct_kwargs = {'add_p': 0,
              'redundancy_amount': 0.25,
              'A': list(np.random.randn(3,p)), 
              'noise_mean': list(np.array([2,3,4,5,1,1,7])),
              'noise_std': list(np.ones(7))}

mu_array = np.array([[1,2,3],[2,1,1]])
sigma_array = 0.5 * np.ones((k,p))

X_splits = []
y_splits = []
for n_ in [50, 100, 100]:
    DataGen = DatasetGenerator(p=p,N=n,K=k,class_task=True,
                               scenario=4, mu_array=mu_array,
                               sigma_array=sigma_array,
                               **dct_kwargs)

    X_splits.append(DataGen.X.T)
    y_splits.append(DataGen.y.T)
    
X_tr, X_vl, X_ts = X_splits
y_tr, y_vl, y_ts = y_splits

del X_splits, y_splits

In [13]:
X_tr.shape, y_tr.shape

((100, 3), (100, 2))

In [29]:
loss_dct = {'square_loss': 'mean_squared_error',
           'cross_entropy': 'categorical_crossentropy'}

nodes = 128
output_dims = 2
loss = 'square_loss'

model = tf.keras.Sequential()
model.add(Dense(nodes, input_shape=(p,), activation='linear'))  # n_params = nodes + bias
    
if loss == 'cross_entropy':
    model.add(Dense(output_dims,
                   activation='softmax'))
elif loss == 'square_loss':
    model.add(Dense(output_dims,
                    activation='softmax'))
    # should we put this activation for the square loss also?
    
sgd = SGD(lr=0.1, momentum=0., nesterov=False)
lr_reduction = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                 patience=5, min_lr=0)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, min_delta=1e-6)

model.compile(optimizer=sgd,
              loss=loss_dct[loss],
              metrics=['accuracy'])

In [30]:
model.build()

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 128)               512       
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 258       
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


In [31]:
X_tr.shape, y_tr.shape

((100, 3), (100, 2))

In [32]:
history = model.fit(X_tr, y_tr, epochs=20)

Train on 100 samples
Epoch 1/20
100/100 [==============================] - 0s 4ms/sample - loss: 0.2245 - accuracy: 0.6000
Epoch 2/20
100/100 [==============================] - 0s 109us/sample - loss: 0.1227 - accuracy: 0.8100
Epoch 3/20
100/100 [==============================] - 0s 391us/sample - loss: 0.0699 - accuracy: 0.9400
Epoch 4/20
100/100 [==============================] - 0s 283us/sample - loss: 0.0515 - accuracy: 0.9900
Epoch 5/20
100/100 [==============================] - 0s 439us/sample - loss: 0.0429 - accuracy: 0.9900
Epoch 6/20
100/100 [==============================] - 0s 774us/sample - loss: 0.0377 - accuracy: 0.9900
Epoch 7/20
100/100 [==============================] - 0s 300us/sample - loss: 0.0343 - accuracy: 0.9900
Epoch 8/20
100/100 [==============================] - 0s 223us/sample - loss: 0.0302 - accuracy: 0.9900
Epoch 9/20
100/100 [==============================] - 0s 200us/sample - loss: 0.0281 - accuracy: 0.9900
Epoch 10/20
100/100 [========================

In [33]:
pd.DataFrame(history.history)

,loss,accuracy
0,0.224550,0.60
1,0.122705,0.81
2,0.069901,0.94
3,0.051538,0.99
4,0.042918,0.99
5,0.037735,0.99
6,0.034322,0.99
7,0.030158,0.99
8,0.028148,0.99
9,0.027971,0.99


In [36]:
?? np.minimum

In [40]:
DataGen.X.shape

(3, 100)

In [88]:
from numpy.linalg import inv

def compute_pinv(x_tr, y_tr, data_gen):
    """ pinv for the different cases
    x_tr must be of size (N, #features)
    y_tr must be of size (N, #classes) """
    n_training, input_dimensions = X.shape
    reduce_dims = False
    original_dims = data_gen.p 
    dct_kwargs = data_gen.dct_kwargs
    dct_kwargs['A'] = np.array(dct_kwargs['A'])
    
    if scenario == 1:
        if input_dimensions > n_training:
            # n < p, ill-posed problem
            inv_val = inv(np.dot(x_tr, x_tr.T))  # this is n x n
            pseudoinv = np.dot(x_tr.T, np.dot(inv_val, y_tr))
        else:
            inv_val = inv(np.dot(x_tr.T, x_tr))  # this is p x p
            pseudoinv = np.dot(inv_val, np.dot(x_tr.T, y_tr))

    elif scenario == 2:
        F = np.vstack((np.identity(original_dims),
                       dct_kwargs['A']))

        if n_training <= original_dims:
            frame_prod = np.dot(F.T, F)
            inv_val = inv(np.dot(np.dot(x_tr[:,:original_dims],
                                        frame_prod),
                                        x_tr[:, :original_dims].T))
            pseudoinv = np.dot(F.dot(x_tr[:, :original_dims].T),
                               np.dot(inv_val, y_tr))
        else:
            # this is wF
            pseudoinv = np.dot(inv(np.dot(x_tr[:, :original_dims].T,
                                          x_tr[:, :original_dims])),
                               np.dot(x_tr[:, :original_dims].T, y_tr))
            reduce_dims = True

    elif scenario == 4:
        dims_redundant_, _ = dct_kwargs['A'].shape
        dims_noise_ = input_dimensions-(dims_redundant_ + original_dims)
        x_tr_ = x_tr[:, :input_dimensions-dims_redundant_]
        lw_part = np.hstack((dct_kwargs['A'],
                             np.zeros((dims_redundant_, dims_noise_))))
        F = np.vstack((np.identity(dims_noise_ + original_dims),
                       lw_part))
        if n_training <= input_dimensions-dims_redundant_:
            inv_val = inv(np.dot(x_tr_, np.dot(np.dot(F.T, F), x_tr_.T)))
            pseudoinv = np.dot(np.dot(F.dot(x_tr_.T), inv_val), y_tr)

        else:  # this is FTW
            pseudoinv = np.dot(inv(np.dot(x_tr_.T, x_tr_)), np.dot(x_tr_.T, y_tr))
            reduce_dims = True
    return pseudoinv, reduce_dims

In [95]:
p = 3
k = 2
n = 100
add_p = 20
scenario = 4
redundancy_amount = 0.20

print('noise', int((1-redundancy_amount)*add_p))
print('redundant', int(add_p * redundancy_amount))

dct_kwargs = {'add_p': add_p,
              'redundancy_amount': 0.25,
              'A': list(np.random.randn(int(add_p * redundancy_amount), p)), 
              'noise_mean': list(np.ones(int((1-redundancy_amount)*add_p))),
              'noise_std': list(np.ones(int((1-redundancy_amount)*add_p)))}

mu_array = np.array([[1,2,3],[2,1,1]]) # np.vstack((np.arange(p), -np.arange(p)))  # 
sigma_array = 0.5 * np.ones((k,p))

DataGen = DatasetGenerator(p=p,N=n,K=k,class_task=True,
                           scenario=scenario, mu_array=mu_array,
                           sigma_array=sigma_array,
                           **dct_kwargs)
X = DataGen.X_transf.T
y = DataGen.y.T

print(X.shape, y.shape)

noise 16
redundant 4
(100, 23) (100, 2)


In [96]:
sol, reduce_flag = compute_pinv(X, y, DataGen)
print(reduce_flag)

True


In [97]:
sol.shape

(19, 2)

In [102]:
x = np.random.randn(10,2)

x

array([[ 0.78894994,  0.63720865],
       [ 0.99129518, -2.18313463],
       [ 0.19475716,  1.22927215],
       [ 0.69974808,  1.68275298],
       [ 1.2434664 , -0.01802449],
       [-0.32192766,  0.03981197],
       [-0.39756698,  0.46471529],
       [-1.03282662,  0.80397278],
       [-0.50260782,  0.96825997],
       [-0.70430505,  1.39876852]])

In [129]:
y_ = np.zeros_like(x).astype('int')
print(y_)

y_[np.arange(y_.shape[0]), np.argmax(x, axis=-1)]=1
print(y_)

y_true = y_.copy()
y_true[0] = np.array([0,1])

print(y_true)

[[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]]
[[1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]
[[0 1]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]


In [133]:
np.sum(np.array([np.dot(y_t, y_p) for (y_t, y_p) in zip(y_true, y_)])) / y_true.shape[0]

0.9

In [131]:
[np.dot(y_t, y_p) for (y_t, y_p) in zip(y_true, y_)]

[0, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [132]:
y_true, y_

(array([[0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1]]), array([[1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1]]))